In [1]:
import pandas as pd
import requests # 웹사이트에 직접 접근
from bs4 import BeautifulSoup
from time import sleep

In [2]:
#1. 웹(html) 문서 불러오기
#clien 요청 -> requests.get('http://url주소')
#              response = headers + content     <- server 응답
# response(응답)은 headers + content으로 구성되어 있다.
# response.headers 는 응답의 http 헤더 내용이 dict 형태로 조회
# response.content 는 http 바디의 내용을 binary(그림,압축,문서) 코드로 조회
# response.text    는 http 바디의 내용을 text(텍스트, http문서) 문자로 조회
# response.encoding은 http 바디가 어떤 인코딩 방식으로 만들어졌는지 조회

url = 'http://192.168.50.54/index.html'
response = requests.get(url)
print(response)
print(response.headers)
#print(response.content)
print(response.encoding)
response.encoding = 'utf-8'
print(response.text)

<Response [200]>
{'Content-Type': 'text/html', 'Last-Modified': 'Sun, 03 Mar 2024 05:28:33 GMT', 'Accept-Ranges': 'bytes', 'ETag': '"85844da22b6dda1:0"', 'Server': 'Microsoft-IIS/10.0', 'Date': 'Sat, 09 Mar 2024 00:59:18 GMT', 'Content-Length': '943'}
ISO-8859-1
<html>
	<head>
		<title>테이블의 구조</title>
		<meta charset="UTF-8" />
		<style>
			table {border:1px solid #000;}
			table th {color:red; background-color: #dcdcdc; border:1px solid #ff0000;}
			table td {border:1px solid #0094ff;}
		</style>
	</head>
	<body>
		다음은 테이블의 예입니다.
		<table>
			<thead>
				<tr>
					<th>1번째 필드</th>
					<th>2번째 필드</th>
					<th>3번째 필드</th>
				</tr>
			</thead>
			<tbody>
				<tr>
					<td>1행_1번째</td>
					<td>1행_2번째</td>
					<td>1행_3번째</td>
				</tr>
				<tr>
					<td>2행_1번째</td>
					<td>2행_2번째</td>
					<td>2행_3번째</td>
				</tr>
				<tr>
					<td>3행_1번째</td>
					<td>3행_2번째</td>
					<td>3행_3번째</td>
				</tr>
				<tr>
					<td>4행_1번째</td>
					<td>4행_2번째</td>
					<td>4행_3번째</td>
				</tr>
			</tbod

In [3]:
#2. 웹문서의 내용(str, 문자열)을 분석가능한 객체(bs4.BeautifulSoup) 생성(변수는 tag로 지정)
#3. find, find_all 메소드 등을 이용하여 특정한 <tag> 블럭 조회
text = response.text
html = BeautifulSoup(text, 'html.parser')

tr_list = html.find('table').find('tbody').find_all('tr')

rows=[]
for tr in tr_list:
    td_list = tr.find_all('td')
    row = []
    for td in td_list:
        data = td.text
        row.append(data)
    rows.append(row)
rows

[['1행_1번째', '1행_2번째', '1행_3번째'],
 ['2행_1번째', '2행_2번째', '2행_3번째'],
 ['3행_1번째', '3행_2번째', '3행_3번째'],
 ['4행_1번째', '4행_2번째', '4행_3번째']]

In [4]:
#웹에서 직접 갖고 오기
url = 'http://192.168.50.54/professors.html'
response = requests.get(url)
#response = headers + content(text.jpeg)
print(response.headers)
text = response.text
print(text[:100])

{'Content-Type': 'text/html', 'Content-Encoding': 'gzip', 'Last-Modified': 'Mon, 07 Mar 2022 23:16:00 GMT', 'Accept-Ranges': 'bytes', 'ETag': '"0b8af4e7932d81:0"', 'Vary': 'Accept-Encoding', 'Server': 'Microsoft-IIS/10.0', 'Date': 'Sat, 09 Mar 2024 00:59:18 GMT', 'Content-Length': '2727'}
<html>
	<head>
		<title>This is jsoup test page</title>

		<script src="add/jquery-1.10.2.min.js


In [5]:
html = BeautifulSoup(text,'html.parser')

In [6]:
print(len(html.find_all('tr')))
tr_list = html.tbody.find_all('tr')

rows = []
for tr in tr_list:
    no = tr.find('td',{'class':'number'}).text
    name = tr.find('td',{'class':'professor'}).text
    lecture = tr.find('td',{'class':'lecture'}).text
    row = [no,name,lecture]
    rows.append(row)
    print(row)
    
pd.DataFrame(rows)

7
['1', 'John', 'C Language']
['2', 'Kim', 'JAVA']
['3', 'Gun', 'PHP / Python']
['4', 'Hun', 'Soccer']
['5', 'Coner', 'Smart Car']


,0,1,2
0,1,John,C Language
1,2,Kim,JAVA
2,3,Gun,PHP / Python
3,4,Hun,Soccer
4,5,Coner,Smart Car


In [7]:
tr = tr_list[0]
print(tr.find_all('td')[0])
print(tr.find_all('td')[1])
# 이 방법은 좋은 방법이 아니다. 
# find('tag',{'attr':'value'})     :<tag attr='value'>...</tag>으로 구성된 태그블럭중 
#                                   가장 처음 발견된 블럭을 반환한다.
# find_all('tag',{'attr':'value'}) :<tag attr='value'>...</tag>으로 구성된 태그블럭들을 
#                                   모두 묶어서 목록으로 반환한다.
# find('td')
#   -> <td class='number'>1</td>
# find('td', {'class':'number'}) 다 못썼음. 아래 실제 코드를 볼 것
#   -> <td class='number'>1</td>
# find('td')
#   -> <td class='number'>1</td>  다 못썼음.
print()
print(tr.find('td'))
print()
print(tr.find('td',{'class':'number'}))
print(tr.find('td',{'class':'professor'}))
print(tr.find('td',{'class':'lecture'}))

<td class="number">1</td>
<td class="professor">John</td>

<td class="number">1</td>

<td class="number">1</td>
<td class="professor">John</td>
<td class="lecture">C Language</td>


In [8]:
#20240309

In [9]:
tr = tr_list[0]
tr.find('td',{'class':'grade'})

<td class="grade hidden-xs">2</td>

In [10]:
grade = tr.find('td',{'class':'grade'}).text

'2'

In [11]:
evaluation = tr.find('td',{'class':'evaluation'}).text

In [12]:
evaluation

'1P'

In [13]:
html = BeautifulSoup(text, 'html.parser')
tr_list = html.find('tbody').find_all('tr')
rows = []
for tr in tr_list:
    no = tr.find('td', {'class':'number'}).text
    name = tr.find('td', {'class':'professor'}).text
    lecture = tr.find('td', {'class':'lecture'}).text
    grade = tr.find('td', {'class':'grade'}).text
    eval = tr.find('td', {'class':'evaluation'}).text
    row = [no, name, lecture]
    rows.append(row)
pd.DataFrame(rows)

,0,1,2
0,1,John,C Language
1,2,Kim,JAVA
2,3,Gun,PHP / Python
3,4,Hun,Soccer
4,5,Coner,Smart Car


In [ ]:
tr = tr_list[0]
grade = tr.find('td', {'class':'grade'}).text
evl = tr.find('td', {'class':'evaluation'}).text
